In [3]:
# Módulos necesarios
!pip install scikit-learn
import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from scipy.stats import expon, reciprocal

# Clasificadores
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [4]:
# Cargamos datos de csv
data = pd.read_csv('../train.csv') # set de entrenamiento
test = pd.read_csv('../test.csv') # set de test

In [5]:
data.describe()

,age,sex,cp,restecg,label
count,732.000000,732.000000,732.000000,732.000000,732.000000
mean,53.364754,0.789617,3.250000,0.602459,1.132514
std,9.306868,0.407859,0.923363,0.802966,1.257615
min,28.000000,0.000000,1.000000,0.000000,0.000000
25%,47.000000,1.000000,3.000000,0.000000,0.000000
50%,54.000000,1.000000,4.000000,0.000000,1.000000
75%,60.000000,1.000000,4.000000,1.000000,2.000000
max,76.000000,1.000000,4.000000,2.000000,4.000000


In [6]:
test.describe()

,age,sex,cp
count,184.000000,184.000000,184.000000
mean,54.179348,0.782609,3.277174
std,9.747938,0.413596,0.937571
min,29.000000,0.000000,1.000000
25%,47.750000,1.000000,3.000000
50%,54.500000,1.000000,4.000000
75%,61.000000,1.000000,4.000000
max,77.000000,1.000000,4.000000


In [7]:
# Quitamos los duplicados y comprobamos
data = data.drop_duplicates(keep = False)
data = data.drop_duplicates(keep = False)
data.describe()

,age,sex,cp,restecg,label
count,730.000000,730.000000,730.000000,730.000000,730.000000
mean,53.376712,0.791781,3.253425,0.604110,1.135616
std,9.316814,0.406313,0.922301,0.803446,1.257937
min,28.000000,0.000000,1.000000,0.000000,0.000000
25%,47.000000,1.000000,3.000000,0.000000,0.000000
50%,54.000000,1.000000,4.000000,0.000000,1.000000
75%,60.000000,1.000000,4.000000,1.000000,2.000000
max,76.000000,1.000000,4.000000,2.000000,4.000000


In [8]:
test.describe()

,age,sex,cp
count,184.000000,184.000000,184.000000
mean,54.179348,0.782609,3.277174
std,9.747938,0.413596,0.937571
min,29.000000,0.000000,1.000000
25%,47.750000,1.000000,3.000000
50%,54.500000,1.000000,4.000000
75%,61.000000,1.000000,4.000000
max,77.000000,1.000000,4.000000


In [9]:
# Buscamos valores únicos en cada variable:

print(f'\nTRAIN SET\n')

for column in data.columns:
    valores_unicos = data[column].unique()
    print(f"Valores únicos en la columna '{column}': {valores_unicos}")

print(f'\nTEST SET\n')

for column in test.columns:
    valores_unicos = test[column].unique()
    print(f"Valores únicos en la columna '{column}': {valores_unicos}")



TRAIN SET

Valores únicos en la columna 'age': [51. 54. 63. 52. 55. 44. 35. 62. 68. 50. 38. 64. 56. 61. 60. 59. 45. 53.
 43. 47. 41. 69. 49. 66. 57. 46. 67. 58. 42. 31. 39. 65. 37. 48. 40. 36.
 33. 70. 34. 76. 71. 29. 32. 74. 73. 72. 30. 75. 28.]
Valores únicos en la columna 'sex': [1. 0.]
Valores únicos en la columna 'cp': [1. 3. 4. 2.]
Valores únicos en la columna 'trestbps': ['125.0' '120.0' '140' '140.0' '122.0' '135' '100' '130.0' '120' '104.0'
 '124' '96' '?' '160.0' '110.0' '134' '134.0' '125' '160' '146.0' '128.0'
 '105' '190' '108.0' '115' '115.0' '150' '112' '100.0' '135.0' '150.0'
 '178' '132.0' '145.0' '130' '110' '170.0' '155' '138.0' '144.0' '180.0'
 '95' '112.0' '122' '105.0' '190.0' '155.0' '170' '117.0' '142' '192.0'
 '114.0' '142.0' '158' '148.0' '172' '124.0' '154' '128' '152.0' '145'
 '118.0' '106.0' '165.0' '132' '172.0' '114' '144' '102.0' '113.0' '174.0'
 '126' '180' '146' '126.0' '154.0' '116' '127' '185' '200.0' '136.0' '118'
 '165' '136' '94.0' '152' '104' '0

In [10]:
# Reemplazamos los valores no imputados ('?', valores no posibles donde corresponda)

# Reemplazamos '?' por NaN en todos los dataset

data.replace('?', np.nan, inplace=True)
test.replace('?', np.nan, inplace=True)

# Reemplazamos '0' por NaN en las columnas 'trestbps', 'chol', 'slope' y 'thal'

# Especificamos las columnas donde reemplazar
columns_to_replace = ['trestbps', 'chol', 'slope', 'thal']
all_columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']

# Reemplazamos '0', '0.0' por NaN
for column in columns_to_replace:
    # Set de entrenamiento
    data[column] = data[column].replace([0, '0', 0.0, '0.0'], np.nan)
    
    # Set de test
    test[column] = test[column].replace([0, '0', 0.0, '0.0'], np.nan)

# Reemplazamos todos los valores negativos en los dataset por NaN
for column in all_columns:
    # Set de entrenamiento
    data[column] = data[column].astype(float)
    data[column] = data[column].apply(lambda x: np.nan if x < 0 else x)
    # Set de test
    test[column] = test[column].astype(float)
    test[column] = test[column].apply(lambda x: np.nan if x < 0 else x)

# Finalmente, llenamos todas las entradas vacías con NaN

# Set de entrenamiento
data.replace(" ", np.nan, inplace=True)
data.replace("", np.nan, inplace=True)
# Set de test
test.replace(" ", np.nan, inplace=True)
test.replace("", np.nan, inplace=True)



In [11]:
# Comprobamos datasets
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 731
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       730 non-null    float64
 1   sex       730 non-null    float64
 2   cp        730 non-null    float64
 3   trestbps  682 non-null    float64
 4   chol      576 non-null    float64
 5   fbs       664 non-null    float64
 6   restecg   730 non-null    float64
 7   thalach   686 non-null    float64
 8   exang     686 non-null    float64
 9   oldpeak   671 non-null    float64
 10  slope     485 non-null    float64
 11  ca        253 non-null    float64
 12  thal      353 non-null    float64
 13  label     730 non-null    int64  
dtypes: float64(13), int64(1)
memory usage: 85.5 KB


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       184 non-null    float64
 1   sex       184 non-null    float64
 2   cp        184 non-null    float64
 3   trestbps  172 non-null    float64
 4   chol      139 non-null    float64
 5   fbs       161 non-null    float64
 6   restecg   182 non-null    float64
 7   thalach   173 non-null    float64
 8   exang     173 non-null    float64
 9   oldpeak   169 non-null    float64
 10  slope     123 non-null    float64
 11  ca        56 non-null     float64
 12  thal      80 non-null     float64
dtypes: float64(13)
memory usage: 18.8 KB


In [13]:
# Comprobamos entradas por cada valor único en cada dataset

print(f'\nTRAINING SET\n')
for column in all_columns:
    print(data[column].value_counts(dropna = False))

print(f'\nTEST SET\n')
for column in all_columns:
    print(test[column].value_counts(dropna = False))


TRAINING SET

54.0    38
58.0    36
56.0    34
57.0    33
52.0    29
55.0    29
59.0    29
53.0    27
51.0    27
62.0    26
61.0    25
41.0    24
63.0    24
60.0    24
48.0    23
50.0    21
49.0    20
46.0    20
43.0    19
64.0    17
42.0    15
44.0    15
65.0    14
45.0    14
47.0    13
39.0    13
38.0    12
66.0    12
69.0    12
37.0    10
67.0     9
68.0     9
35.0     8
40.0     7
36.0     6
70.0     5
34.0     5
71.0     4
74.0     4
32.0     3
75.0     3
72.0     2
31.0     2
29.0     2
76.0     2
73.0     1
30.0     1
33.0     1
28.0     1
Name: age, dtype: int64
1.0    578
0.0    152
Name: sex, dtype: int64
4.0    391
3.0    168
2.0    136
1.0     35
Name: cp, dtype: int64
120.0    109
130.0     88
140.0     78
NaN       48
150.0     42
110.0     42
160.0     41
125.0     24
135.0     17
128.0     15
100.0     14
115.0     14
170.0     12
112.0     12
122.0     11
145.0     11
124.0     11
180.0     11
132.0     10
118.0      9
134.0      9
138.0      9
105.0      7
142.0     

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 731
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       730 non-null    float64
 1   sex       730 non-null    float64
 2   cp        730 non-null    float64
 3   trestbps  682 non-null    float64
 4   chol      576 non-null    float64
 5   fbs       664 non-null    float64
 6   restecg   730 non-null    float64
 7   thalach   686 non-null    float64
 8   exang     686 non-null    float64
 9   oldpeak   671 non-null    float64
 10  slope     485 non-null    float64
 11  ca        253 non-null    float64
 12  thal      353 non-null    float64
 13  label     730 non-null    int64  
dtypes: float64(13), int64(1)
memory usage: 85.5 KB


In [15]:
# Ahora, probamos estrategias para inputar valores en los NaNs

# Inicializamos SimpleImputer
imputer = SimpleImputer(strategy='mean')

# Hacemos fit a SimpleImputer y computamos
data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
test = pd.DataFrame(imputer.fit_transform(test), columns=test.columns)

In [16]:
# En este paso, normalizamos los datos

scaler = StandardScaler()
data[all_columns] = scaler.fit_transform(data[all_columns])
test[all_columns] = scaler.fit_transform(test[all_columns])

In [17]:
# Aplicamos one-hot encoding a las variables categóricas
data = pd.get_dummies(data)
test = pd.get_dummies(test)


In [18]:
# Supongamos que tienes un DataFrame df y deseas dividirlo en conjuntos de entrenamiento y validación
X = data.drop('label', axis=1)  # Features (variables independientes)
y = data['label']  # Variable objetivo (variable dependiente)

# Dividir el DataFrame en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)
X_test = test

# test_size=0.2 indica que el 20% de los datos se utilizarán para validación
# random_state=42 establece una semilla aleatoria para reproducibilidad


In [19]:
# Crear un clasificador k-NN
knn = KNeighborsClassifier(n_neighbors=5)  # Define el número de vecinos, puedes ajustarlo según sea necesario

# Entrenar el clasificador con los datos de entrenamiento
knn.fit(X_train, y_train)

# Hacer predicciones en los datos de validación
y_pred = knn.predict(X_val)
from sklearn.metrics import accuracy_score, confusion_matrix

# Calcular la precisión del modelo
precisionknn = accuracy_score(y_val, y_pred)
print("Precisión del modelo:", precisionknn)

# Calcular la matriz de confusión
matriz_confusionknn = confusion_matrix(y_val, y_pred)
print("Matriz de confusión:")
print(matriz_confusionknn)

Precisión del modelo: 0.5272727272727272
Matriz de confusión:
[[45  9  2  0  1]
 [ 7  6  4  4  1]
 [ 1  2  4  1  2]
 [ 5  3  4  3  1]
 [ 2  0  3  0  0]]


In [20]:
# Crear un clasificador Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)  # Puedes ajustar el número de árboles (n_estimators)

# Entrenar el clasificador con los datos de entrenamiento
random_forest.fit(X_train, y_train)

# Hacer predicciones en los datos de validación
y_pred_rf = random_forest.predict(X_val)



# Calcular la precisión del modelo
precisionrf = accuracy_score(y_val, y_pred_rf)
print("Precisión del modelo:", precisionrf)

# Calcular la matriz de confusión
matriz_confusionrf = confusion_matrix(y_val, y_pred_rf)
print("Matriz de confusión:")
print(matriz_confusionrf)


Precisión del modelo: 0.5818181818181818
Matriz de confusión:
[[49  5  3  0  0]
 [ 8  9  3  1  1]
 [ 1  1  4  4  0]
 [ 3  2  8  2  1]
 [ 0  2  1  2  0]]


In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Crear un clasificador de árbol de decisión
decision_tree = DecisionTreeClassifier(max_depth=4, random_state=42)  # Puedes ajustar los parámetros aquí

# Entrenar el clasificador con los datos de entrenamiento
decision_tree.fit(X_train, y_train)

# Hacer predicciones en los datos de validación
y_pred_dt = decision_tree.predict(X_val)

# Hacer predicciones en los datos de test
y_pred_test_dt = decision_tree.predict(X_test)

# Calcular la precisión del modelo
precisiondt = accuracy_score(y_val, y_pred_dt)
print("Precisión del modelo:", precisiondt)

# Calcular la matriz de confusión
matriz_confusiondt = confusion_matrix(y_val, y_pred_dt)
print("Matriz de confusión:")
print(matriz_confusiondt)


Precisión del modelo: 0.5909090909090909
Matriz de confusión:
[[52  3  1  1  0]
 [ 6  8  4  4  0]
 [ 3  1  2  4  0]
 [ 4  4  5  3  0]
 [ 0  1  2  2  0]]


In [ ]:
# Random Forest con RandomSearch

# Definimos los parámetros del grid
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Crear un clasificador Random Forest
random_forest = RandomForestClassifier()

# Inicializamos RandomizedSearchCV
random_search = RandomizedSearchCV(random_forest, param_distributions=param_dist, n_iter=1000, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit RandomizedSearchCV con datos de entrenamiento
random_search.fit(X_train, y_train)

# Mejores parámetros y score
best_params = random_search.best_params_
best_score = random_search.best_score_

print("Mejores parámetros:", best_params)
print("Mejore score:", best_score)

# Predición con mejores parámetros, set de validación
y_pred_val = random_search.predict(X_val)

# Predición con mejores parámetros, set de test
y_pred_test_rf = random_search.predict(X_test)

# Hacer predicciones en los datos de validación
accuracy = accuracy_score(y_val, y_pred_val)
print("Accuracy on test data:", accuracy)

Fitting 3 folds for each of 540 candidates, totalling 1620 fits


/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 540 is smaller than n_iter=1000. Running 540 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.3s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, min_sam

In [ ]:
# Random Forest con GridSearch

# Definimos los parámetros del grid
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False]  # Method for sampling data points
}

# Crear un clasificador Random Forest
random_forest = RandomForestClassifier()

# Inicializamos RandomizedSearchCV
grid_search = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv=3, verbose=2, n_jobs=-1)


# Fit RandomizedSearchCV con datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mejores parámetros y score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Mejores parámetros:", best_params)
print("Mejore score:", best_score)

# Predición con mejores parámetros, set de validación
y_pred_val = grid_search.predict(X_val)

# Predición con mejores parámetros, set de test
y_pred_test_rf = grid_search.predict(X_test)

# Hacer predicciones en los datos de validación
accuracy = accuracy_score(y_val, y_pred_val)
print("Accuracy on test data:", accuracy)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; tota

In [ ]:
# Crear un clasificador Naive Bayes
naive_bayes = GaussianNB()

# Entrenar el clasificador con los datos de entrenamiento
naive_bayes.fit(X_train, y_train)

# Hacer predicciones en los datos de validación
y_pred_nb = naive_bayes.predict(X_val)

# Calcular la precisión del modelo
precisionnb = accuracy_score(y_val, y_pred_nb)
print("Precisión del modelo:", precisionnb)

# Calcular la matriz de confusión
matriz_confusionnb = confusion_matrix(y_val, y_pred_nb)
print("Matriz de confusión:")
print(matriz_confusionnb)


Precisión del modelo: 0.6181818181818182
Matriz de confusión:
[[45 11  1  0  0]
 [ 4 12  2  4  0]
 [ 0  2  4  3  1]
 [ 3  3  1  6  3]
 [ 0  1  0  3  1]]


In [ ]:
# Crear un clasificador Gradient Boosting
gradient_boosting = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)  
# Puedes ajustar el número de árboles (n_estimators) y la tasa de aprendizaje (learning_rate)

# Entrenar el clasificador con los datos de entrenamiento
gradient_boosting.fit(X_train, y_train)

# Hacer predicciones en los datos de validación
y_pred_gb = gradient_boosting.predict(X_val)

precisiongb = accuracy_score(y_val, y_pred_gb)
print("Precisión del modelo:", precisiongb)

# Calcular la matriz de confusión
matriz_confusionnb = confusion_matrix(y_val, y_pred_gb)
print("Matriz de confusión:")
print(matriz_confusionnb)


Precisión del modelo: 0.5636363636363636
Matriz de confusión:
[[46  3  2  5  1]
 [ 6  8  3  3  2]
 [ 1  0  5  3  1]
 [ 3  4  4  3  2]
 [ 0  0  4  1  0]]


In [ ]:
# Crear un clasificador de regresión logística
logistic_regression = LogisticRegression(random_state=42)  

# Entrenar el clasificador con los datos de entrenamiento
logistic_regression.fit(X_train, y_train)

# Hacer predicciones en los datos de validación
y_pred_lr = logistic_regression.predict(X_val)

precisionrl = accuracy_score(y_val, y_pred_lr)
print("Precisión del modelo:", precisionrl)

# Calcular la matriz de confusión
matriz_confusionrl = confusion_matrix(y_val, y_pred_lr)
print("Matriz de confusión:")
print(matriz_confusionrl)


Precisión del modelo: 0.5636363636363636
Matriz de confusión:
[[48  8  0  1  0]
 [ 6  7  2  7  0]
 [ 1  3  2  4  0]
 [ 3  5  1  5  2]
 [ 0  1  0  4  0]]


In [ ]:
# Crear un clasificador SVM
svm_classifier = SVC(kernel='linear', random_state=42)  # Kernel lineal, puedes cambiarlo según sea necesario

# Entrenar el clasificador con los datos de entrenamiento
svm_classifier.fit(X_train, y_train)

# Hacer predicciones en los datos de validación
y_pred_svm = svm_classifier.predict(X_val)

# Hacer predicciones en los datos de test
y_test = svm_classifier.predict(X_test)

precisionsvm = accuracy_score(y_val, y_pred_svm)
print("Precisión del modelo, validación:", precisionsvm)


# Calcular la matriz de confusión (validación)
matriz_confusionsvm = confusion_matrix(y_val, y_pred_svm)
print("Matriz de confusión:")
print(matriz_confusionsvm)


Precisión del modelo, validación: 0.6272727272727273
Matriz de confusión:
[[50  7  0  0  0]
 [ 5 10  2  5  0]
 [ 1  3  3  3  0]
 [ 3  5  2  6  0]
 [ 0  1  1  3  0]]


In [ ]:
# VSM con RandomSearch

# Definimos la distribución de parametros
param_dist = {
    'C': reciprocal(1, 1000),
    'gamma': expon(scale=1.0),
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

# Crear un clasificador SVM
svm_classifier = SVC()

# Configuración RandomizedSearchCV
random_search = RandomizedSearchCV(svm_classifier, param_distributions=param_dist, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Best parameters and score
best_params = random_search.best_params_
best_score = random_search.best_score_

print("Mejores parámetros:", best_params)
print("Mejore score:", best_score)

# Predicción en set de validación con mejores parámetros
y_pred_val = random_search.predict(X_val)

# Predicción en set de test con mejores parámetros
y_pred_test_vsm = random_search.predict(X_test)

# Evaluaaos el modelo (set de validación)
accuracy = accuracy_score(y_val, y_pred_val)
print("Accuracy on test data:", accuracy)


Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END C=13.292918943162162, gamma=3.010121430917521, kernel=poly; total time=   0.0s
[CV] END C=13.292918943162162, gamma=3.010121430917521, kernel=poly; total time=   0.0s
[CV] END C=13.292918943162162, gamma=3.010121430917521, kernel=poly; total time=   0.0s
[CV] END C=218.30968390524592, gamma=0.9084469696321253, kernel=rbf; total time=   0.0s
[CV] END C=2.937538457632828, gamma=0.059838768608680676, kernel=sigmoid; total time=   0.0s
[CV] END C=218.30968390524592, gamma=0.9084469696321253, kernel=rbf; total time=   0.0s
[CV] END C=10.025956902289563, gamma=0.15416196746656105, kernel=poly; total time=   0.0s
[CV] END C=2.937538457632828, gamma=0.059838768608680676, kernel=sigmoid; total time=   0.0s
[CV] END C=2.937538457632828, gamma=0.059838768608680676, kernel=sigmoid; total time=   0.0s
[CV] END C=218.30968390524592, gamma=0.9084469696321253, kernel=rbf; total time=   0.0s
[CV] END C=10.025956902289563, gamma=0.1

In [38]:
# Montamos el DF para el generar el archivo csv para Kaggle
# Por ahora, RF con RandomSearch a 540 iteraciones ha dado el mejor resultado

prediccion = pd.DataFrame()
prediccion['ID'] = range(len(test))
prediccion['label'] = y_pred_test_dt.astype(int)

prediccion.head()

prediccion.to_csv('0804a_dt.csv', index = False)